In [93]:
%matplotlib inline

import numpy as np
import random
from graph_generator import kronecker_random_graph
from cascade import generate_cascade

In [102]:
def generate_cascade(g):
    g = g.copy()
    source = random.choice(g.nodes())
    infected = {source}
    infected_times = {source: 0}
    iter_n = 0
    while True:
        iter_n += 1
        newly_infected = set()
        for u in infected:
            can_continue = False
            for v in g.neighbors(u):
                if v not in infected and not g[u][v].get('attempted', False):
                    can_continue = True
                    p_uv = 0.5  # proba of getting infected                                                                                                   
                    if random.random() < p_uv:
                        newly_infected.add(v)
                        infected_times[v] = iter_n
                    g[u][v]['attempted'] = True
        infected |= newly_infected
        if not can_continue:
            break
    return infected_times

In [156]:
P_peri = np.array([[0.9, 0.1], [0.1, 0.3]])

In [194]:
def run_simulation(P, k, n_edges):
    g = kronecker_random_graph(k, P, directed=False, n_edges=n_edges)
    g = g.subgraph(max(nx.connected_components(g), key=len))

    percentage = 0.2

    infected_times = None
    for _ in range(100):
        infected_times = generate_cascade(g)
        if len(infected_times) > 10:
            break

    assert infected_times is not None

    source = min(infected_times, key=lambda k: infected_times[k])
    infected_minus_source = set(infected_times.keys()) - {source}

    observed = {n: infected_times[n]
                for n in random.sample(infected_minus_source,
                                       int(len(infected_minus_source) * percentage))}
    assert source not in observed
    
    # BFS-style searching
    current_node = min(observed, key=lambda k: observed[k])
    query_n = 0
    while True:
        # print('current eariliest infected node {} ({})'.format(current_node, observed[current_node]))
        queries = [n for n in g.neighbors(current_node)
                   if n not in observed]
        query_n += len(queries)
        newly_found_infections = {q: infected_times.get(q)
                                  for q in queries 
                                  if infected_times.get(q) is not None}
        newly_found_earlier_infections = [q
                                          for q in newly_found_infections
                                          if infected_times[q] < infected_times[current_node]]
        if len(newly_found_earlier_infections) == 0:
            break
        else:
            observed.update(newly_found_infections)        
            current_node = min(newly_found_earlier_infections, key=lambda k: observed[k])

    assert current_node == source, '{}({}) !== {}({})'.format(current_node, infected_times[current_node],
                                                              source, infected_times[source])
    return g, infected_times, query_n

In [195]:
from joblib import Parallel, delayed
rows = Parallel(n_jobs=8)(delayed(run_simulation)(P_peri, 8, 512) for _ in range(100))


In [198]:
data = map(lambda t: (len(t[1]), t[2]), rows)
df = pd.DataFrame(list(data), columns=['csize', 'qsize'])

In [201]:
cs = df['csize'].describe()
qs = df['qsize'].describe()
rs = (df['qsize'] / df['csize']).describe()

stat = pd.DataFrame([cs.as_matrix(), qs.as_matrix(), rs.as_matrix()],
                    columns=rs.index,
                    index=['cascade size', 'query size', 'query size / cascade size'])

In [202]:
stat

,count,mean,std,min,25%,50%,75%,max
cascade size,100.0,68.520000,37.230779,11.000000,33.750000,72.500000,101.250000,128.000000
query size,100.0,18.170000,9.535743,3.000000,9.750000,18.000000,26.000000,40.000000
query size / cascade size,100.0,0.439443,0.553603,0.025424,0.164474,0.253247,0.416915,3.454545
